# ML.Net - BagOfWords

In [1]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML" 

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.ML version 1.5.0

## Using C# Class

In [2]:
using Microsoft.ML;
using Microsoft.ML.Data;
using System;
using System.Collections.Generic;

## Declare data-classes for input data and predictions

In [3]:
public class Output
{
    public float[] BagOfWords { get; set; }
}
    
internal class Input
{
    public string Text { get; set; }
}

In [4]:
var context = new MLContext();

var data = new List<Input>
{
    new Input { Text = "I really enjoy programming in C#. And I really like it." }
};

var dataView = context.Data.LoadFromEnumerable(data);

var bagWordsPipeline = context.Transforms.Text.ProduceWordBags(
        "BagOfWords",
        "Text",
        ngramLength: 1,
        useAllLengths: false,
        weighting: Microsoft.ML.Transforms.Text.NgramExtractingEstimator.WeightingCriteria.Tf
    );

var bagWordsTransform = bagWordsPipeline.Fit(dataView);
var bagWordsDataView = bagWordsTransform.Transform(dataView);

var predictionEngine = context.Model.CreatePredictionEngine<Input, Output>(bagWordsTransform);

var prediction = predictionEngine.Predict(data[0]);

VBuffer<ReadOnlyMemory<char>> slotNames = default;
bagWordsDataView.Schema["BagOfWords"].GetSlotNames(ref slotNames);

var bagOfWordColumn = bagWordsDataView.GetColumn<VBuffer<float>>(bagWordsDataView.Schema["BagOfWords"]);
var slots = slotNames.GetValues();

Console.Write("NGrams: ");
foreach (var featureRow in bagOfWordColumn)
{
    foreach (var item in featureRow.Items())
    {
        Console.Write($"{slots[item.Key]}  ");
    }

    Console.WriteLine();
}

Console.Write("Word Counts: ");
for (int i = 0; i < prediction.BagOfWords.Length; i++)
{
    Console.Write($"{prediction.BagOfWords[i]:F4}  ");
}

Console.WriteLine(Environment.NewLine);

Unhandled exception: (29,1): error CS8345: O campo ou a propriedade autoimplementada não pode ser do tipo 'ReadOnlySpan<ReadOnlyMemory<char>>', a menos que seja um membro de instância de uma struct de referência.